# 🇺🇸 10-Year USA Financial Data Collection - Google Colab Edition

**Purpose**: Collect 10 years of USA financial events from GDELT for forecasting research

**Features**:
- ✅ Automatic Google Drive integration
- ✅ Resume capability if interrupted
- ✅ Real-time progress tracking
- ✅ Runs independently of your laptop
- ✅ USA-focused financial filtering

**Estimated Runtime**: 4-6 hours
**Expected Output**: ~5,000 high-quality USA financial events (2015-2025)

## 🔧 Setup & Installation

In [ ]:
# Install required packages
!pip install gdelt pandas numpy

# Import libraries
import gdelt
import pandas as pd
import numpy as np
import os
import time
from datetime import datetime, timedelta
from google.colab import drive, files
import json

print("✅ All packages installed successfully!")

## 📁 Google Drive Setup

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Create project folder in Google Drive
project_folder = '/content/drive/MyDrive/Financial_Forecasting_Data'
os.makedirs(project_folder, exist_ok=True)

# Set up file paths
raw_data_file = os.path.join(project_folder, 'gdelt_usa_10year_raw.csv')
progress_file = os.path.join(project_folder, 'collection_progress.json')
log_file = os.path.join(project_folder, 'collection_log.txt')

print(f"📁 Project folder created: {project_folder}")
print(f"📄 Raw data will be saved to: {raw_data_file}")
print(f"📊 Progress tracking: {progress_file}")
print(f"📝 Logs will be saved to: {log_file}")

## ⚙️ Configuration & Filtering Setup

In [ ]:
# Initialize GDELT
gd = gdelt.gdelt(version=2)

# 10-year date range
START_DATE = "2015-01-01"
END_DATE = "2025-01-01"

# USA-specific filtering criteria
USA_LOCATIONS = [
    'United States', 'New York', 'Manhattan', 'Wall Street', 'NYSE', 'NASDAQ',
    'Washington', 'Federal Reserve', 'Fed', 'Treasury', 'SEC', 
    'California', 'Silicon Valley', 'San Francisco', 'Los Angeles',
    'Chicago', 'Illinois', 'Boston', 'Massachusetts', 'Texas', 'Florida',
    'White House', 'Congress', 'Senate', 'House of Representatives'
]

USA_FINANCIAL_THEMES = [
    'ECON_STOCKMARKET', 'ECON_INFLATION', 'ECON_RECESSION', 'ECON_GDP',
    'ECON_UNEMPLOYMENT', 'ECON_INTEREST_RATE', 'EPU_ECONOMY', 'EPU_POLICY',
    'WB_1920_FINANCIAL_SECTOR_DEVELOPMENT', 'WB_439_MACROECONOMIC_AND_STRUCTURAL_POLICIES'
]

US_FINANCIAL_SOURCES = [
    'reuters', 'bloomberg', 'wsj', 'cnbc', 'marketwatch', 'yahoo',
    'cnn', 'foxnews', 'abc', 'nbc', 'cbs', 'ap.org', 'usatoday',
    'washingtonpost', 'nytimes', 'fortune', 'forbes'
]

# Create filter patterns
usa_location_pattern = '|'.join(USA_LOCATIONS)
usa_theme_pattern = '|'.join(USA_FINANCIAL_THEMES)
usa_source_pattern = '|'.join(US_FINANCIAL_SOURCES)

print(f"🇺🇸 USA Location Filter: {len(USA_LOCATIONS)} keywords")
print(f"🏷️ Financial Theme Filter: {len(USA_FINANCIAL_THEMES)} themes")
print(f"📰 News Source Filter: {len(US_FINANCIAL_SOURCES)} sources")
print(f"📅 Date Range: {START_DATE} to {END_DATE}")

## 🛠️ Utility Functions

In [ ]:
def log_message(message):
    """Log message to both console and file"""
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    log_entry = f"[{timestamp}] {message}"
    print(log_entry)
    
    # Save to log file
    with open(log_file, 'a', encoding='utf-8') as f:
        f.write(log_entry + '\n')

def save_progress(stats, completed_periods):
    """Save progress to JSON file"""
    progress_data = {
        'last_updated': datetime.now().isoformat(),
        'completed_periods': completed_periods,
        'stats': stats
    }
    
    with open(progress_file, 'w') as f:
        json.dump(progress_data, f, indent=2)

def load_progress():
    """Load progress from JSON file"""
    if os.path.exists(progress_file):
        try:
            with open(progress_file, 'r') as f:
                return json.load(f)
        except:
            return None
    return None

def parse_v2tone_quick(v2tone_str):
    """Quick V2Tone parsing for sentiment scores"""
    try:
        parts = str(v2tone_str).split(',')
        if len(parts) >= 7:
            return {
                'tone_avg': float(parts[0]),
                'tone_positive': float(parts[1]),
                'tone_negative': float(parts[2]),
                'word_count': int(parts[6])
            }
    except:
        pass
    return {'tone_avg': 0, 'tone_positive': 0, 'tone_negative': 0, 'word_count': 0}

def keep_alive():
    """Prevent Colab from disconnecting during long runs"""
    from IPython.display import Javascript
    display(Javascript('''
        function ClickConnect(){
            console.log("Working");
            document.querySelector("colab-toolbar-button#connect").click();
        }
        setInterval(ClickConnect,60000);
    '''))

print("✅ Utility functions loaded")

## 🔄 Resume Capability Check

In [ ]:
# Check for existing data and resume point
monthly_ranges = pd.date_range(start=START_DATE, end=END_DATE, freq='MS')
resume_from = 0
existing_data = []

# Load previous progress
progress_data = load_progress()
if progress_data:
    log_message(f"📂 Found previous progress from {progress_data['last_updated']}")
    resume_from = progress_data.get('completed_periods', 0)
    log_message(f"🔄 Will resume from period {resume_from + 1}/{len(monthly_ranges)}")

# Check for existing raw data
if os.path.exists(raw_data_file):
    try:
        existing_df = pd.read_csv(raw_data_file)
        if not existing_df.empty:
            existing_data.append(existing_df)
            log_message(f"📄 Found existing data with {len(existing_df)} events")
        else:
            log_message("📄 Found empty data file")
    except Exception as e:
        log_message(f"⚠️ Could not read existing data file: {e}")
else:
    log_message("📄 No existing data file found - starting fresh")

# Initialize collection statistics
collection_stats = {
    'total_periods': len(monthly_ranges),
    'completed_periods': resume_from,
    'total_raw_events': 0,
    'total_usa_events': len(existing_data[0]) if existing_data else 0,
    'failed_periods': 0,
    'start_time': datetime.now().isoformat()
}

log_message(f"📊 Collection will process {len(monthly_ranges) - resume_from} remaining periods")
log_message(f"⏱️ Estimated remaining time: {(len(monthly_ranges) - resume_from) * 5 / 60:.1f} hours")

## 🔌 Keep Colab Alive (Run this to prevent disconnection)

In [ ]:
# Keep Colab session alive during long runs
keep_alive()
log_message("🔌 Keep-alive activated - Colab will stay connected")

## 🚀 Main Data Collection Loop

In [ ]:
# Main data collection loop
all_usa_events = existing_data.copy()

log_message("🇺🇸 Starting 10-year USA financial data collection...")
log_message(f"📅 Period: {START_DATE} to {END_DATE}")
log_message(f"📊 Total periods to collect: {len(monthly_ranges)}")

try:
    for i, month_start in enumerate(monthly_ranges[resume_from:], start=resume_from):
        # Calculate period end (end of month)
        if month_start.month == 12:
            month_end = month_start.replace(year=month_start.year + 1, month=1, day=1) - timedelta(days=1)
        else:
            month_end = month_start.replace(month=month_start.month + 1, day=1) - timedelta(days=1)
        
        month_start_str = month_start.strftime('%Y-%m-%d')
        month_end_str = month_end.strftime('%Y-%m-%d')
        
        log_message(f"📅 Period {i+1}/{len(monthly_ranges)}: {month_start_str} to {month_end_str}")
        
        try:
            # Search GDELT for the current month
            results_df = gd.Search(
                date=[month_start_str, month_end_str],
                table='gkg',
                output='df'
            )
            
            if not results_df.empty:
                raw_count = len(results_df)
                collection_stats['total_raw_events'] += raw_count
                
                # Apply USA location filter first (most restrictive)
                usa_location_filter = results_df['Locations'].str.contains(
                    usa_location_pattern, case=False, na=False
                )
                usa_located = results_df[usa_location_filter].copy()
                
                if not usa_located.empty:
                    # Apply USA financial theme filter
                    usa_theme_filter = usa_located['Themes'].str.contains(
                        usa_theme_pattern, case=False, na=False, regex=True
                    )
                    usa_themed = usa_located[usa_theme_filter].copy()
                    
                    if not usa_themed.empty:
                        # Apply USA source filter
                        usa_source_filter = usa_themed['SourceCommonName'].str.contains(
                            usa_source_pattern, case=False, na=False
                        )
                        usa_final = usa_themed[usa_source_filter].copy()
                        
                        if not usa_final.empty:
                            usa_count = len(usa_final)
                            collection_stats['total_usa_events'] += usa_count
                            all_usa_events.append(usa_final)
                            
                            log_message(f"  ✅ Raw: {raw_count} → USA Financial: {usa_count} events")
                        else:
                            log_message(f"  📰 No events after USA source filtering")
                    else:
                        log_message(f"  🏷️ No events after USA theme filtering")
                else:
                    log_message(f"  📍 No USA located events")
            else:
                log_message(f"  ❌ No events found for this period")
                
        except Exception as e:
            log_message(f"  ⚠️ Error collecting data for this period: {e}")
            collection_stats['failed_periods'] += 1
            continue
        
        collection_stats['completed_periods'] += 1
        
        # Save progress every 6 months
        if (i + 1) % 6 == 0 and all_usa_events:
            log_message(f"💾 Saving progress at {month_start.year}...")
            temp_df = pd.concat(all_usa_events, ignore_index=True)
            temp_df.to_csv(raw_data_file, index=False)
            save_progress(collection_stats, collection_stats['completed_periods'])
            log_message(f"   Saved {len(temp_df)} total USA events so far")
        
        # Be respectful to GDELT servers
        time.sleep(4)  # 4 seconds between requests for Colab
        
        # Progress update every 3 months
        if (i + 1) % 3 == 0:
            elapsed_hours = (datetime.now() - pd.to_datetime(collection_stats['start_time'])).total_seconds() / 3600
            remaining_periods = len(monthly_ranges) - collection_stats['completed_periods']
            estimated_remaining = (remaining_periods * elapsed_hours) / max(collection_stats['completed_periods'], 1)
            
            log_message(f"📊 Progress Update:")
            log_message(f"   Completed: {collection_stats['completed_periods']}/{collection_stats['total_periods']} periods")
            log_message(f"   USA Events Collected: {collection_stats['total_usa_events']}")
            log_message(f"   Failed Periods: {collection_stats['failed_periods']}")
            log_message(f"   Elapsed Time: {elapsed_hours:.1f} hours")
            log_message(f"   Estimated Remaining: {estimated_remaining:.1f} hours")

except KeyboardInterrupt:
    log_message("⏹️ Collection interrupted by user")
except Exception as e:
    log_message(f"💥 Unexpected error: {e}")

log_message("🏁 Collection phase complete!")

## 🎯 Final Data Processing & Analysis

In [ ]:
# Final data processing
log_message("🎯 Starting final data processing...")

if all_usa_events:
    # Combine all collected data
    final_usa_df = pd.concat(all_usa_events, ignore_index=True)
    
    # Remove duplicates
    initial_count = len(final_usa_df)
    final_usa_df = final_usa_df.drop_duplicates(subset=['DATE', 'DocumentIdentifier'])
    duplicates_removed = initial_count - len(final_usa_df)
    
    log_message(f"🔄 Removed {duplicates_removed} duplicate events")
    
    # Parse dates and add time features
    final_usa_df['date_parsed'] = pd.to_datetime(final_usa_df['DATE'], format='%Y%m%d%H%M%S', errors='coerce')
    final_usa_df['date_only'] = final_usa_df['date_parsed'].dt.date
    final_usa_df['year'] = final_usa_df['date_parsed'].dt.year
    final_usa_df['month'] = final_usa_df['date_parsed'].dt.month
    final_usa_df['weekday'] = final_usa_df['date_parsed'].dt.weekday
    final_usa_df['hour'] = final_usa_df['date_parsed'].dt.hour
    
    # Parse sentiment scores
    log_message("📊 Parsing sentiment scores...")
    sentiment_data = final_usa_df['V2Tone'].apply(parse_v2tone_quick)
    sentiment_df = pd.json_normalize(sentiment_data)
    
    for col in sentiment_df.columns:
        final_usa_df[col] = sentiment_df[col].values
    
    # Save final processed data
    final_usa_df.to_csv(raw_data_file, index=False)
    
    # Create summary statistics
    log_message("📈 Generating summary statistics...")
    
    # Yearly breakdown
    yearly_counts = final_usa_df.groupby('year').size()
    monthly_avg = final_usa_df.groupby(['year', 'month']).size().groupby('year').mean()
    
    # Sentiment statistics
    sentiment_stats = final_usa_df['tone_avg'].describe()
    
    # Source breakdown
    top_sources = final_usa_df['SourceCommonName'].value_counts().head(10)
    
    # Save summary report
    summary_file = os.path.join(project_folder, 'collection_summary.txt')
    with open(summary_file, 'w') as f:
        f.write("🇺🇸 USA Financial Data Collection - Final Report\n")
        f.write("=" * 50 + "\n\n")
        f.write(f"Collection Period: {START_DATE} to {END_DATE}\n")
        f.write(f"Total Events Collected: {len(final_usa_df):,}\n")
        f.write(f"Average Events per Year: {len(final_usa_df) / len(yearly_counts):.0f}\n")
        f.write(f"Date Range in Data: {final_usa_df['date_only'].min()} to {final_usa_df['date_only'].max()}\n\n")
        
        f.write("Yearly Distribution:\n")
        for year, count in yearly_counts.items():
            f.write(f"  {year}: {count:,} events\n")
        
        f.write(f"\nSentiment Statistics:\n")
        f.write(f"  Mean: {sentiment_stats['mean']:.2f}\n")
        f.write(f"  Std: {sentiment_stats['std']:.2f}\n")
        f.write(f"  Min: {sentiment_stats['min']:.2f}\n")
        f.write(f"  Max: {sentiment_stats['max']:.2f}\n")
        
        f.write(f"\nTop 10 News Sources:\n")
        for source, count in top_sources.items():
            f.write(f"  {source}: {count:,} events\n")
    
    # Final statistics
    collection_stats['end_time'] = datetime.now().isoformat()
    collection_stats['final_event_count'] = len(final_usa_df)
    collection_stats['duplicates_removed'] = duplicates_removed
    
    # Calculate total runtime
    start_time = pd.to_datetime(collection_stats['start_time'])
    end_time = pd.to_datetime(collection_stats['end_time'])
    total_hours = (end_time - start_time).total_seconds() / 3600
    
    log_message("\n" + "=" * 50)
    log_message("🏆 COLLECTION COMPLETE - FINAL RESULTS")
    log_message("=" * 50)
    log_message(f"📊 Total USA Financial Events: {len(final_usa_df):,}")
    log_message(f"📅 Date Range: {final_usa_df['date_only'].min()} to {final_usa_df['date_only'].max()}")
    log_message(f"⏱️ Total Runtime: {total_hours:.2f} hours")
    log_message(f"📈 Average Events per Year: {len(final_usa_df) / len(yearly_counts):.0f}")
    log_message(f"💾 Data saved to: {raw_data_file}")
    log_message(f"📋 Summary saved to: {summary_file}")
    
    log_message("\n📅 Yearly Distribution:")
    for year, count in yearly_counts.items():
        log_message(f"   {year}: {count:,} events")
    
    log_message("\n✅ Ready for financial forecasting model development!")
    
    # Save final progress
    save_progress(collection_stats, collection_stats['completed_periods'])
    
else:
    log_message("❌ No data collected - check your filters and API connectivity")

log_message("🎯 Final processing complete!")

## 📥 Download Results (Optional)

In [ ]:
# Optional: Download files to your local machine
# (Data is already saved to Google Drive)

try:
    if os.path.exists(raw_data_file):
        log_message("📥 Downloading data file...")
        files.download(raw_data_file)
    
    if os.path.exists(summary_file):
        log_message("📥 Downloading summary file...")
        files.download(summary_file)
    
    if os.path.exists(log_file):
        log_message("📥 Downloading log file...")
        files.download(log_file)
        
    log_message("✅ Download complete!")
    
except Exception as e:
    log_message(f"⚠️ Download failed: {e}")
    log_message("💡 Files are still available in Google Drive")

## 📋 Instructions for Use

### 🚀 **To Run This Notebook**:
1. **Run all cells in order** (Ctrl+F9 or Runtime → Run all)
2. **Authorize Google Drive** when prompted
3. **Keep the browser tab open** (the keep-alive function will help)
4. **Check progress** in the Google Drive folder periodically

### 🔄 **If Interrupted**:
- Simply re-run the notebook
- It will automatically resume from where it left off
- Progress is saved every 6 months

### 📁 **Output Files** (in Google Drive):
- `gdelt_usa_10year_raw.csv` - Main dataset
- `collection_summary.txt` - Statistics and summary
- `collection_log.txt` - Detailed logs
- `collection_progress.json` - Progress tracking

### ⏱️ **Expected Timeline**:
- **Setup**: 2-3 minutes
- **Data Collection**: 4-6 hours
- **Final Processing**: 5-10 minutes
- **Total**: ~5-7 hours

### 🎯 **Expected Results**:
- ~5,000 high-quality USA financial events
- Perfect for S&P 500, NASDAQ forecasting
- Ready for machine learning models

### 💡 **Tips**:
- **Colab Pro** recommended for longer runtimes
- Can run overnight without your laptop
- Files automatically sync to Google Drive
- Resume capability if connection drops